In [1]:
import pandas as pd
import numpy as np

In [2]:
#Benign without Sample

In [17]:
df_at = pd.read_csv("complete/part-00000-1c1962bd-22d7-4427-a10a-76ffb17d853e-c000.csv")

In [18]:
df_at.shape
#(471454, 85)

(2830743, 23)

In [20]:
def transform(x):
    if x == "BENIGN":
        return 1
    else:
        return 0

In [22]:
df_at['nLabel'] = df_at.Label.map(lambda x : transform(x))

In [23]:
import pickle
with open('labelDict.bin','rb') as fout:
    dd = pickle.load(fout)

featureList =[x for sublist in list(dd.values())  for x in sublist]

colList= list(set(featureList))
colList.append("nLabel")

In [10]:
colList

['Bwd Packets/s',
 'Subflow Fwd Bytes',
 'Init_Win_bytes_forward',
 'Fwd IAT Min',
 'Fwd PSH Flags',
 'Total Length of Fwd Packets',
 'ACK Flag Count',
 'Flow IAT Min',
 'SYN Flag Count',
 'PSH Flag Count',
 'Fwd Packet Length Mean',
 'Fwd Packets/s',
 'Bwd IAT Mean',
 'Fwd IAT Mean',
 'Active Min',
 'Flow IAT Mean',
 'Average Packet Size',
 'Active Mean',
 'Bwd Packet Length Std',
 'Flow IAT Std',
 'Flow Duration',
 'Init_Win_bytes_backward',
 'Label']

In [24]:
need_df = df_at[colList]

In [25]:
need_df.shape

(2830743, 23)

In [26]:
from sklearn.model_selection import train_test_split

y= need_df.pop('nLabel')
X_train, X_validtest, y_train, y_validtest = train_test_split(need_df, y, test_size=0.20, random_state=42,stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_validtest, y_validtest, test_size=0.50, random_state=42,stratify=y_validtest)

In [36]:
np.savetxt('multiclass-tensor/withoutSample/benignAttack/X_train.np',X_train.values)
np.savetxt('multiclass-tensor/withoutSample/benignAttack/y_train.np',y_train.values)

np.savetxt('multiclass-tensor/withoutSample/benignAttack/X_test.np',X_test.values)
np.savetxt('multiclass-tensor/withoutSample/benignAttack/y_test.np',y_test.values)

np.savetxt('multiclass-tensor/withoutSample/benignAttack/X_valid.np',X_valid.values)
np.savetxt('multiclass-tensor/withoutSample/benignAttack/y_valid.np',y_valid.values)

In [27]:
# class weightage
y_test.value_counts()

1    235929
0     47146
Name: nLabel, dtype: int64

In [29]:
val = df_at.groupby('nLabel').count().iloc[:,0]

In [30]:
val.values.tolist()

[471454, 2359289]

In [31]:
val.index.tolist()

[0, 1]

In [30]:
df_at.groupby('Label').count().iloc[:,0]

Label
Bot                           1966
DDoS                         41835
DoS GoldenEye                10293
DoS Hulk                    231073
DoS Slowhttptest              5499
DoS slowloris                 5796
FTP-Patator                   7938
Heartbleed                      11
Infiltration                    36
PortScan                    158930
SSH-Patator                   5897
Web Attack-Brute Force        1507
Web Attack-Sql Injection        21
Web Attack-XSS                 652
Name: Flow ID, dtype: int64

In [12]:
pd.get_dummies(y_test)[:5]

,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack-Brute Force,Web Attack-Sql Injection,Web Attack-XSS
2163253,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1251089,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1256009,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1930814,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2620756,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [32]:
labeldict = { k:v for k,v in zip([_ for _ in range(len(val.values.tolist()))],val.values.tolist())}

In [14]:
labeldict

{0: 2359289,
 1: 1966,
 2: 41835,
 3: 10293,
 4: 231073,
 5: 5499,
 6: 5796,
 7: 7938,
 8: 11,
 9: 36,
 10: 158930,
 11: 5897,
 12: 1507,
 13: 21,
 14: 652}

In [15]:
import math
def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()

    for key in keys:
        #print(total)
        #print(type(labels_dict[key]))
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight

In [33]:
create_class_weight(labeldict)

{0: 1.0, 1: 1.0}

In [49]:
#Attack with Sampling

In [37]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 0.8)
x_res, y_res = sm.fit_sample(X_train, y_train)

/Users/gauravvashisth/anaconda3/envs/envtf/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [38]:
type(y_res)

numpy.ndarray

In [39]:
y_res[:5]

array([1, 1, 1, 1, 0])

In [40]:
pd.get_dummies(y_train).values

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [41]:
x_res.values()

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [43]:
np.savetxt('multiclass-tensor/withSample/benignAttack/X_train.np',x_res)
np.savetxt('multiclass-tensor/withSample/benignAttack/y_train.np',y_res)

np.savetxt('multiclass-tensor/withSample/benignAttack/X_test.np',X_test.values)
np.savetxt('multiclass-tensor/withSample/benignAttack/y_test.np',y_test)

np.savetxt('multiclass-tensor/withSample/benignAttack/X_valid.np',X_valid.values)
np.savetxt('multiclass-tensor/withSample/benignAttack/y_valid.np',y_valid)

In [44]:
y_res

array([1, 1, 1, ..., 0, 0, 0])